<a href="https://colab.research.google.com/github/iued-uni-heidelberg/corpustools/blob/main/europarlAnalysis2023V01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysing Europarl corpora: directionality in translation
## Discovering translation parameter in original vs. translated corpora

In [ ]:
%%bash
# dowloading datasets
wget https://heibox.uni-heidelberg.de/f/a1cb95b59a404c1998ab/?dl=1
mv index.html?dl=1 ep_org_de.tgz
wget https://heibox.uni-heidelberg.de/f/ad00fed92da841198943/?dl=1
mv index.html?dl=1 ep_org_en.tgz
wget https://heibox.uni-heidelberg.de/f/b4913557be5b44dba797/?dl=1
mv index.html?dl=1 ep_trans_de_en.tgz
wget https://heibox.uni-heidelberg.de/f/7d1f642beb434d4a8878/?dl=1
mv index.html?dl=1 ep_trans_en_de.tgz

## Original and translation corpora

In [ ]:
%%bash
ls ./

In [ ]:
%%bash
tar xvzf ep_org_de.tgz
tar xvzf ep_org_en.tgz
tar xvzf ep_trans_de_en.tgz
tar xvzf ep_trans_en_de.tgz

In [4]:
%%bash
cat ep_org_de/* >ep_org_de.vert
cat ep_org_en/* >ep_org_en.vert
cat ep_trans_de_en/* >ep_trans_de_en.vert
cat ep_trans_en_de/* >ep_trans_en_de.vert


In [5]:
%%bash
wc ep_org_de.vert
wc ep_org_en.vert
wc ep_trans_de_en.vert
wc ep_trans_en_de.vert


 4616795 13890492 83988042 ep_org_de.vert
 4657686 14017453 71587447 ep_org_en.vert
 5161989 15526775 78117513 ep_trans_de_en.vert
 4568156 13747601 84879715 ep_trans_en_de.vert


In [ ]:
%%bash
wc *.vert


In [ ]:
!head -n 40 ep_org_de.vert

In [11]:
# selecting column 1 (for lemmatization with the TreeTagger)
!awk -F '\t' '{if($0 ~ /^</){printf "\n%s\n", $0}else if(NF==3){j++; printf "%s ", $1}} END{print j+0  > "/dev/stderr" }' <ep_org_de.vert >ep_org_de.txt
!awk -F '\t' '{if($0 ~ /^</){printf "\n%s\n", $0}else if(NF==3){j++; printf "%s ", $1}} END{print j+0  > "/dev/stderr" }' <ep_org_en.vert >ep_org_en.txt
!awk -F '\t' '{if($0 ~ /^</){printf "\n%s\n", $0}else if(NF==3){j++; printf "%s ", $1}} END{print j+0  > "/dev/stderr" }' <ep_trans_de_en.vert >ep_trans_de_en.txt
!awk -F '\t' '{if($0 ~ /^</){printf "\n%s\n", $0}else if(NF==3){j++; printf "%s ", $1}} END{print j+0  > "/dev/stderr" }' <ep_trans_en_de.vert >ep_trans_en_de.txt

4450075
4474780
4995111
4385848


In [ ]:
!head -n 40 ep_trans_en_de.txt

In [ ]:
# selecting column 3 (for using original lemmatization)
# !awk -F '\t' '{if($0 ~ /^</){printf "%s\n", $0}else if(NF==3){j++; printf "%s ", $3}} END{print j+0  > "/dev/stderr" }' <ep_org_de.vert >ep_org_de_lem.txt
!awk -F '\t' '{if($0 ~ /^</){printf "%s\n", $0}else if(NF==3){j++; printf "%s ", $3}} END{print j+0  > "/dev/stderr" }' <ep_org_de.vert >ep_org_de_lem0.txt
!awk -F '\t' '{if($0 ~ /^</){printf "%s\n", $0}else if(NF==3){j++; printf "%s ", $3}} END{print j+0  > "/dev/stderr" }' <ep_org_en.vert >ep_org_en_lem0.txt
!awk -F '\t' '{if($0 ~ /^</){printf "%s\n", $0}else if(NF==3){j++; printf "%s ", $3}} END{print j+0  > "/dev/stderr" }' <ep_trans_de_en.vert >ep_trans_de_en_lem0.txt
!awk -F '\t' '{if($0 ~ /^</){printf "%s\n", $0}else if(NF==3){j++; printf "%s ", $3}} END{print j+0  > "/dev/stderr" }' <ep_trans_en_de.vert >ep_trans_en_de_lem0.txt

4450075


In [ ]:
!head -n 40 /content/ep_org_de_lem.txt
!wc /content/ep_org_de_lem.txt

## Lemmatization with TreeTagger (English, German)

In [ ]:
%%bash
mkdir treetagger
cd treetagger
# Download the tagger package for your system (PC-Linux, Mac OS-X, ARM64, ARMHF, ARM-Android, PPC64le-Linux).
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tree-tagger-linux-3.2.4.tar.gz
tar -xzvf tree-tagger-linux-3.2.4.tar.gz
# Download the tagging scripts into the same directory.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/tagger-scripts.tar.gz
gunzip tagger-scripts.tar.gz
# Download the installation script install-tagger.sh.
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/install-tagger.sh
# Download the parameter files for the languages you want to process.
# list of all files (parameter files) https://cis.lmu.de/~schmid/tools/TreeTagger/#parfiles
wget https://cis.lmu.de/~schmid/tools/TreeTagger/data/english.par.gz
sh install-tagger.sh
cd ..
sudo pip install treetaggerwrapper
# changing options: no-unknown, sgml, lemma
mv /content/treetagger/cmd/tree-tagger-english /content/tree-tagger-english0
awk '{ if (NR == 9) print "OPTIONS=\"-token -lemma -sgml -no-unknown\""; else print $0}' /content/tree-tagger-english0 > /content/treetagger/cmd/tree-tagger-english
chmod a+x ./treetagger/cmd/tree-tagger-english

# downloading German
wget https://heibox.uni-heidelberg.de/f/ec8226edebb64a359407/?dl=1
mv index.html?dl=1 /content/treetagger/lib/german-utf8.par

# German2 = -no-unknown 
# note: the default "tree-tagger-german" command file will not work, as parameter files have not been downloaded, only use tree-tagger-german2
wget https://heibox.uni-heidelberg.de/f/acb9b8a2fa4f40e08f8a/?dl=1
mv index.html?dl=1 /content/treetagger/cmd/tree-tagger-german2

chmod a+x /content/treetagger/cmd/tree-tagger-german2


In [ ]:
!./treetagger/cmd/tree-tagger-english <ep_org_en.txt >ep_org_en_lem1TT.txt
!./treetagger/cmd/tree-tagger-english <ep_trans_de_en.txt >ep_trans_de_en_lem1TT.txt

!./treetagger/cmd/tree-tagger-german2 <ep_org_de.txt >ep_org_de_lem1TT.txt
!./treetagger/cmd/tree-tagger-german2 <ep_trans_en_de.txt >ep_trans_en_de_lem1TT.txt
